In [120]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

def load_and_clean():
    """
    spotify, identify = load_and_clean()
    """
    spotify = pd.read_csv('SpotifyFeatures.csv')

    # dataframe that serves to identify songs
    identify = spotify[['artist_name', 'track_id', 'track_name']]

    # dataframe consisting of audio features we want to train on
    spotify = spotify.drop(columns = ['genre',
                                    'mode',
                                    'time_signature',
                                    'key',
                                    'track_id',
                                    'artist_name',
                                    'popularity',
                                    'track_name',
                                    'duration_ms',
                                    'speechiness'])

    # genre_map = {'Movie': 0, 'R&B': 1, 'A Capella': 2, 'Alternative': 3, 'Country': 4, 'Dance': 5, 'Electronic': 6, 'Anime': 7, 'Folk': 8, 'Blues': 9, 'Opera': 10, 'Hip-Hop': 11, "Children's Music": 12, 'Children’s Music': 12, 'Rap': 13, 'Indie': 14, 'Classical': 15, 'Pop': 16, 'Reggae': 17, 'Reggaeton': 18, 'Jazz': 19, 'Rock': 20, 'Ska': 21, 'Comedy': 22, 'Soul': 23, 'Soundtrack': 24, 'World': 25}

    # spotify['genre'] = spotify['genre'].map(genre_map)

    return spotify, identify

spotify, identify = load_and_clean()
print(spotify.shape)
print(spotify.head())
print('-----------------')
print(identify.shape)
print(identify.head())

(232725, 8)
   acousticness  danceability  energy  instrumentalness  liveness  loudness  \
0         0.611         0.389   0.910             0.000    0.3460    -1.828   
1         0.246         0.590   0.737             0.000    0.1510    -5.559   
2         0.952         0.663   0.131             0.000    0.1030   -13.879   
3         0.703         0.240   0.326             0.000    0.0985   -12.178   
4         0.950         0.331   0.225             0.123    0.2020   -21.150   

     tempo  valence  
0  166.969    0.814  
1  174.003    0.816  
2   99.488    0.368  
3  171.758    0.227  
4  140.576    0.390  
-----------------
(232725, 3)
         artist_name                track_id                        track_name
0     Henri Salvador  0BRjO6ga9RKCKjfDqeFgWV       C'est beau de faire un Show
1  Martin & les fées  0BjC1NfoEOOusryehmNudP  Perdu d'avance (par Gad Elmaleh)
2    Joseph Williams  0CoSDzoNIKCRs124s9uTVy    Don't Let Me Be Lonely Tonight
3     Henri Salvador  0Gc6TVm52BwZD

In [121]:
spotify.isnull().sum(0)

acousticness        0
danceability        0
energy              0
instrumentalness    0
liveness            0
loudness            0
tempo               0
valence             0
dtype: int64

In [122]:
spotify.head()

,acousticness,danceability,energy,instrumentalness,liveness,loudness,tempo,valence
0,0.611,0.389,0.910,0.000,0.3460,-1.828,166.969,0.814
1,0.246,0.590,0.737,0.000,0.1510,-5.559,174.003,0.816
2,0.952,0.663,0.131,0.000,0.1030,-13.879,99.488,0.368
3,0.703,0.240,0.326,0.000,0.0985,-12.178,171.758,0.227
4,0.950,0.331,0.225,0.123,0.2020,-21.150,140.576,0.390


In [123]:
from sklearn.cluster import KMeans
import json

In [104]:
def neural_net():
    '''
    insert here
    '''

    n_clusters = 


SyntaxError: invalid syntax (<ipython-input-104-45ea9ffefa2a>, line 6)

In [156]:
def knn_predictor(audio_feats, k=20):
    """
    similar_song_ids, visual_df = knn_predictor(audio_features)
    """
    # Scale the data with standard scaler
    scaler = StandardScaler()
    spotify_scaled = scaler.fit_transform(spotify)

    ################################################
    audio_feats_scaled = scaler.transform([audio_feats])

    ## Nearest Neighbors model
    knn = NearestNeighbors(n_neighbors=k, algorithm='kd_tree')
    knn.fit(spotify_scaled)

    # prediction 
    prediction = knn.kneighbors(audio_feats_scaled)

    # # Get the indexes of the list of similar songs
    # if prediction[0][0][0] == 0.0:
    #     similar_songs_index = prediction[1][0][1:].tolist()
    # else:
    similar_songs_index = prediction[1][0][:k].tolist()
    
    # Create an empty list to store simlar song names
    similar_song_ids = []
    similar_song_names = []

    # loop over the indexes and append song names to empty list above
    for i in similar_songs_index:
        song_id = identify['track_id'].iloc[i]
        similar_song_ids.append(song_id)
        song_name = identify['track_name'].iloc[i]
        similar_song_names.append(song_name)

    #################################################

    column_names = spotify.columns.tolist()

    # put scaled audio features into a dataframe
    audio_feats_scaled_df = pd.DataFrame(audio_feats_scaled, columns=column_names)

    # create empty list of similar songs' features
    similar_songs_features = []

    # loop through the indexes of similar songs to get audio features for each
    #. similar song
    for index in similar_songs_index:
        list_of_feats = spotify.iloc[index].tolist()
        similar_songs_features.append(list_of_feats)

    # scale the features and turn them into a dataframe
    similar_feats_scaled = scaler.transform(similar_songs_features)
    similar_feats_scaled_df = pd.DataFrame(similar_feats_scaled, columns=column_names)

    

    # get the % difference between the outputs and input songs
    col_names = similar_feats_scaled_df.columns.to_list()
    diff_df = pd.DataFrame(columns=col_names)
    for i in range(k):
        diff = abs(similar_feats_scaled_df.iloc[i] - audio_feats_scaled_df.iloc[0])
        # print('type: ', type(similar_feats_scaled_df.iloc[i]))
        diff_df.loc[i] = diff
    
    # add sums of differences 
    diff_df['sum'] = diff_df.sum(axis=1)
    diff_df = diff_df.sort_values(by=['sum'])
    diff_df = diff_df.reset_index(drop=True)

    # add track_id to DF
    diff_df['track_id'] = similar_song_ids

    # reorder cols to have track_id as first column
    cols = list(diff_df)
    cols.insert(0, cols.pop(cols.index('track_id')))
    diff_df = diff_df.loc[:, cols]
    print(diff_df)

    return similar_song_ids, similar_song_names, visual_df, diff_df


test_audio_features = [0.5, 0.5, 0.5, 0.1, 0.25, -5.0, 125, 0.5]
similar_song_ids, similar_song_names, visual_df, diff_df = knn_predictor(test_audio_features)



diff_json = diff_df.to_json(orient='records')

# print(diff_json)
# print('-----------------')
# print('Recommended song_ids:')
# print(similar_song_ids)
# print('Recommended song_names:')
# print(similar_song_names)
# print('-----------------')
# print(visual_df)

track_id  acousticness  danceability    energy  \
0   48x1VbWiN6kiJAdGBISzhn      0.146575      0.102366  0.106280   
1   2XZt5ACXfEiF9QyxMnwaIr      0.149394      0.043102  0.258109   
2   6rPeI30oZ1yoATa1cDmEFS      0.132481      0.043102  0.026570   
3   3UXQEfdcI6MWoNFUQ2uXYg      0.132481      0.043102  0.026570   
4   0NkceLa3wcYsgtkxzAnSfV      0.132481      0.043102  0.026570   
5   22ciZPW9sugBzMdD97YE2G      0.132481      0.043102  0.026570   
6   3dAOzj0Yb3hHD6sllOKtZR      0.011275      0.161631  0.102484   
7   2iqi3UbkTjkpQJSMII3arQ      0.177581      0.026939  0.083506   
8   5OsXhFFtEXrVV3eaPWFrFA      0.202950      0.096979  0.072119   
9   2x0supcZhd6NEH1nem9DnR      0.056375      0.064652  0.083506   
10  2x0supcZhd6NEH1nem9DnR      0.109931      0.258610  0.018979   
11  2x0supcZhd6NEH1nem9DnR      0.186037      0.070040  0.091097   
12  2x0supcZhd6NEH1nem9DnR      0.163487      0.021551  0.079710   
13  7zpNN01FLMP70EYGcxCCDY      0.408719      0.177794  0.121463  

In [ ]:
# # create empty list for averaged features of recommended songs
    # similar_feats_averaged = []

    # # loop through columns of audio features and get average of each column for 5
    # #. recommended songs
    # for col in column_names:
    #     avg = similar_feats_scaled_df[col].mean()
    #     similar_feats_averaged.append(avg)
    
    # # print('Sum of means of all predicted songs: ', sum(similar_feats_averaged))

    # # turn averages into 1 row dataframe
    # similar_feats_averaged_df = pd.DataFrame([similar_feats_averaged], columns=column_names)

    # # concatenate this with input songs audio features to be used for visualizing
    # visual_df = pd.concat([audio_feats_scaled_df, similar_feats_averaged_df], ignore_index=True)

    